# Dimensión sede

In [24]:
import pandas as pd
import yaml
from sqlalchemy import create_engine
from datetime import date

### Conexión con la base de datos

In [25]:
with open('../config.yml', 'r') as f:
  config = yaml.safe_load(f)
  config_oltp = config['fuente']
  config_olap = config['bodega']

url_oltp = (f"{config_oltp['drivername']}://{config_oltp['user']}:{config_oltp['password']}@{config_oltp['host']}:"f"{config_oltp['port']}/{config_oltp['dbname']}")
url_olap = (f"{config_olap['drivername']}://{config_olap['user']}:{config_olap['password']}@{config_olap['host']}:"f"{config_olap['port']}/{config_olap['dbname']}")
#print(url_oltp)
#print(url_olap)
oltp_conn = create_engine(url_oltp)
olap_conn = create_engine(url_olap)

### Módulo de extracción

In [ ]:
table_sede = pd.read_sql_table('sede', oltp_conn)
table_ciudad = pd.read_sql_table('ciudad', oltp_conn)
table_departamento = pd.read_sql_table('departamento', oltp_conn)

### Módulo de transformación

In [ ]:
table_ciudad = (pd.merge(table_ciudad.rename(columns={'nombre' : 'ciudad'}), table_departamento.rename(columns={'nombre' : 'departamento'}), how='inner')
  .drop(columns=['departamento_id']))

dimension_sede = (pd.merge(table_sede.rename(columns={'nombre' : 'nombre_sede'}), table_ciudad, 
  how='inner', left_on='ciudad_id', right_on='ciudad_id')
  .drop(columns=['ciudad_id', 'cliente_id'])
  .sort_values(by='sede_id'))

dimension_sede.head()

,sede_id,nombre_sede,direccion,telefono,nombre_contacto,ciudad,departamento
14,1,Sede principal - Cliente1,Los angeles distrito Latino,310-70000,JUAN PEREZ,CALI,VALLE DEL CAUCA
15,2,sede aux - cliente 1,Los angeles distrito Latino,310-70000,JUAN PEREZ,CALI,VALLE DEL CAUCA
49,3,TORRES DE MARACAIBO,Los angeles distrito Latino,310-70000,JUAN PEREZ,CALI,VALLE DEL CAUCA
50,4,INGENIO,Los angeles distrito Latino,310-70000,JUAN PEREZ,CALI,VALLE DEL CAUCA
16,5,VASQUEZ COBO,Los angeles distrito Latino,310-70000,JUAN PEREZ,CALI,VALLE DEL CAUCA


### Módulo de carga a la bodega de datos

In [28]:
dimension_sede.set_index('sede_id', inplace=True)
try:
  dimension_sede.to_sql('dim_sede', olap_conn, if_exists='replace')
except Exception as e:
  print(f"Error al cargar datos: {e}")